In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
sns.set()
from sklearn import preprocessing
   

In [2]:
raw_data = pd.read_csv('housing.csv', delim_whitespace=True)

In [3]:
raw_data

,0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98,24.00
0,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
1,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
4,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222.0,18.7,394.12,5.21,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
501,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
502,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
503,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [4]:
columns_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRADIO', 'B-1000', 'LSTAT', 'MEDV']


In [5]:
raw_data.columns = columns_names

In [6]:
raw_data.index = raw_data.index + 1
raw_data.loc[0] = [0.00632, 18.00, 2.310, 0, 0.5380, 6.5750, 65.20, 4.0900, 1, 296.0, 15.30, 396.90, 4.98, 24.00]
raw_data = raw_data.sort_index()
raw_data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRADIO,B-1000,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [7]:
data = raw_data.copy()
data_no_medv_vif = data.drop(['NOX','MEDV'], axis=1)


In [8]:
from sklearn.linear_model import LinearRegression
y = data['MEDV']

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

#crédito a The Data Science Course 2021: Complete Data Science Bootcamp - aula 454
class CustomScaler(BaseEstimator,TransformerMixin): 
    

    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
               
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    

    def transform(self, X, y=None, copy=None):
        
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)        
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]
#a The Data Science Course 2021: Complete Data Science Bootcamp - aula 454 
    
unscaled_inputs = data_no_medv_vif
columns_to_omit = ['CHAS']
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

scaler = CustomScaler(columns_to_scale)
scaler.fit(unscaled_inputs)
scaled_inputs = scaler.transform(unscaled_inputs)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass copy=True, with_mean=True, with_std=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(scaled_inputs,y,train_size=0.8,shuffle=True,random_state=20)

In [10]:
reg = LinearRegression()
reg.fit(x_train,y_train)

LinearRegression()

In [11]:
reg.score(x_train,y_train)

0.7216892430558413

In [12]:
reg.coef_

array([-0.49423159,  0.97137678, -0.27913642,  2.72095383,  2.73355343,
       -0.0696485 , -2.20941043,  1.87961082, -2.10136642, -1.42162045,
        0.97344995, -4.23573154])

In [13]:
reg.intercept_

22.491842858757995

In [14]:
reg_tabela = pd.DataFrame([['Bias'],['CRIM'],['ZN'],['INDUS'],['CHAS'],['RM'],['AGE'],['DIS'],['RAD'],['TAX'],['PTRADIO'],['B-1000'],['LSTAT']],columns=['Features'])
reg_tabela

,Features
0,Bias
1,CRIM
2,ZN
3,INDUS
4,CHAS
5,RM
6,AGE
7,DIS
8,RAD
9,TAX


In [15]:
reg_tabela['Weights'] = reg.intercept_,reg.coef_[0],reg.coef_[1],reg.coef_[2],reg.coef_[3],reg.coef_[4],reg.coef_[5],reg.coef_[6],reg.coef_[7],reg.coef_[8],reg.coef_[9],reg.coef_[10],reg.coef_[11]

In [16]:
reg_tabela


,Features,Weights
0,Bias,22.491843
1,CRIM,-0.494232
2,ZN,0.971377
3,INDUS,-0.279136
4,CHAS,2.720954
5,RM,2.733553
6,AGE,-0.069648
7,DIS,-2.209410
8,RAD,1.879611
9,TAX,-2.101366


In [17]:
reg.score(x_test,y_test)

0.7488795468592077